In [0]:
try:
    import tflearn    
except ImportError:
    !pip3 install -q tflearn
    import tflearn 

In [0]:
!apt-get install -y -q ffmpeg
!pip3 install -q scikit-video
import skvideo.io 

In [0]:
import matplotlib.pyplot as plt 
import numpy as np 
from glob import glob 
import os, sys  
import cv2
import tensorflow as tf  
from tflearn.activations import relu 


from tflearn.layers.normalization import batch_normalization as bn
from scipy.misc import imread, bytescale, imresize  
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d, global_avg_pool
from tflearn.layers.estimator import regression  


from tflearn.data_utils import to_categorical  


from tflearn.data_augmentation import ImageAugmentation


from tensorflow.contrib.tensorboard.plugins import projector

In [0]:
    def install_tensorboard_dep():
        '''Installs tensorboard to be used in colab.'''
        if 'ngrok-stable-linux-amd64.zip' not in os.listdir(os.getcwd()):
            !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
            !unzip ngrok-stable-linux-amd64.zip
            os.system('n')

In [0]:
            def montage(x, plot_shape=(15, 15), return_grid=False, cmap='viridis'):
                '''
                Takes in a 4-D tensor, x, of shape [NxHxWxC] and puts all of the images
                together in a single matrix to show as one image. If return_grid is True, 
                it will not show the matrix, but return it to be used in some other 
                operations.
                '''
                
                count = 0
                if not return_grid:
                    scale = [0, 255]
                else:
                    scale = [0, 1]
                
                
                if len(x.shape) == 4 and x.shape[-1] == 3:
                    num, m, n, c = x.shape
                else:
                    num, m, n = x.shape
                    c = 1
                    
                num = int(np.ceil(np.sqrt(num)))
                grid = np.zeros([num*m, num*n, c])
                
                if c == 1:
                    grid = grid[..., 0]
                
                for i in range(num):
                    for j in range(num):
                        if count < x.shape[0]:
                            if c == 1:
                                grid[i*m:i*m+m, j*n:j*n+n] = bytescale(x[count, ...], 
                                                                                                             low=scale[0], high=scale[1])
                            else:
                                grid[i*m:i*m+m, j*n:j*n+n, :] = bytescale(x[count, ...],
                                                                                                                    low=scale[0], high=scale[1])
                            count += 1 
                            
                if return_grid:
                    return grid
                else:
                    fig = plt.figure(figsize=plot_shape)
                    a1 = fig.add_subplot(111)
                    a1.set_xticks(np.arange(-0.5, num*n, n))
                    a1.set_yticks(np.arange(-0.5, num*m, m))
                    a1.set_yticklabels([])
                    a1.set_xticklabels([])
                    a1.imshow(grid, cmap=cmap)
                    plt.show()

In [0]:
def start_tensorboard():
        '''Starts tensorboard on colaboratory.'''
        LOG_DIR = '/tmp/tflearn_logs'    
        get_ipython().system_raw('tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'.format(LOG_DIR))
        get_ipython().system_raw('./ngrok http 6006 &')
        ! curl -s http://localhost:4040/api/tunnels | python3 -c \
        "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
def residual_block(incoming, n_filters, filter_size, n_blocks, stride=1):
    
    for block in range(n_blocks):
        n, h, w, c = incoming.get_shape().as_list() 
        
        if block > 0 and stride > 1:
            stride = 1
    
        
        conv1 = conv_2d(incoming, n_filters, filter_size, activation='linear', strides=stride)
        conv1_norm = relu(bn(conv1))
    
        
        conv2 = conv_2d(conv1_norm, n_filters, filter_size, activation='linear', strides=1)
        conv2_norm = bn(conv2)
        
        if stride > 1:
            incoming = max_pool_2d(incoming, 3, stride)
        
        if c != n_filters:
            incoming = conv_2d(incoming, n_filters, 1, activation='linear', strides=1)
    
        
        incoming = incoming + conv2_norm
    
      
        incoming = relu(incoming)
        
        return incoming

In [0]:
def video_loader():
    vid_names = glob('*.avi') 
    print(vid_names)
    num_past_frames = 3
    vids = np.zeros([0, 240, 320, num_past_frames])
    
    for indx, vid_name in enumerate(vid_names):
        try:
            vid = skvideo.io.vread(vid_name) 
            print(vid.shape)
        except ValueError:
            continue
       
        if vid.shape[1] != 240 or vid.shape[2] != 320:
            vid = [imresize(vid[f, ...], [240, 320]) for f in range(vid.shape[0])]
     
        vid = np.mean(vid, 3, keepdims=True)    
        
        vid_stack = np.zeros([vid.shape[0]-num_past_frames, vid.shape[1], vid.shape[2], num_past_frames])
        
        for frame_id in range((num_past_frames-1), vid.shape[0]):
            past_1 = vid[frame_id-num_past_frames, ...]
            past_2 = vid[frame_id-2*num_past_frames, ...]
            
            stack = np.concatenate((vid[frame_id, ...], past_1, past_2), 2)
            
            vid_stack[frame_id-num_past_frames, ...] = stack
            
                                                    
        vids = np.concatenate((vids, vid_stack), 0)
        
    return vids 

In [0]:
os.chdir('/home/sofia_lynch/Documents/')

# Input training data

In [0]:
os.chdir('climb_stairs')

In [0]:
imgs_train = video_loader()

In [0]:
climbstairs_group_imgs_train = imgs_train.shape[0]
print(imgs_train.shape)

In [0]:
labels_train = np.zeros([climbstairs_group_imgs_train, ]) 
print(labels_train.shape)

In [0]:
labels_train[climbstairs_group_imgs_train:] = 0. 

In [0]:
climb_stairs_img_train = imgs_train[0, ...]    
for i in range(3):    
    plt.imshow(climb_stairs_img_train[..., i], cmap='gray')    
    plt.grid(False) 
    plt.show()

In [0]:
os.chdir('..') 
os.chdir('run') 

In [0]:
imgs_train = np.concatenate((imgs_train, video_loader()), 0) 

In [0]:
run_group_imgs_train = imgs_train.shape[0]
print(imgs_train.shape)

In [0]:
labels_train = np.zeros([run_group_imgs_train, ]) 
print(labels_train.shape)

In [0]:
labels_train[climbstairs_group_imgs_train:run_group_imgs_train] = 1. 

In [0]:
run_img_train = imgs_train[0, ...]        
for i in range(3):       
        plt.imshow(run_img_train[..., i], cmap='gray')        
        plt.grid(False) 
        plt.show()

In [0]:
os.chdir('..') 
os.chdir('ride_bike') 

In [0]:
imgs_train = np.concatenate((imgs_train, video_loader()), 0) 

In [0]:
ridebike_group_imgs_train = imgs_train.shape[0]
print(imgs_train.shape)

In [0]:
labels_train = np.zeros([ridebike_group_imgs_train, ]) 
print(labels_train.shape)

In [0]:
labels_train[run_group_imgs_train:ridebike_group_imgs_train] = 2. 

In [0]:
ride_bike_img_train = imgs_train[0, ...]       
for i in range(3):       
        plt.imshow(ride_bike_img_train[..., i], cmap='gray')       
        plt.grid(False) 
        plt.show()

In [0]:
print(imgs_train.shape)

In [0]:
from tflearn.data_utils import to_categorical 
labels_train = to_categorical(labels_train, 3) 

In [0]:
print(labels_train.shape)

# Input Test Data

In [0]:
os.chdir('..')
os.chdir('climb_stairs_test')

In [0]:
imgs_test = video_loader()

In [0]:
climbstairs_group_imgs_test = imgs_test.shape[0]
print(imgs_test.shape)

In [0]:
labels_test = np.zeros([climbstairs_group_imgs_test, ])
print(labels_test.shape)

In [0]:
labels_test[climbstairs_group_imgs_test:] = 0. 

In [0]:
climb_stairs_img_test = imgs_test[0, ...]        
for i in range(3):       
        plt.imshow(climb_stairs_img_test[..., i], cmap='gray')        
        plt.grid(False) 
        plt.show()

In [0]:
os.chdir('..') 
os.chdir('run_test') 

In [0]:
imgs_test = np.concatenate((imgs_test, video_loader()), 0) 

In [0]:
run_group_imgs_test = imgs_test.shape[0]
print(imgs_test.shape)

In [0]:
labels_test = np.zeros([run_group_imgs_test, ]) 
print(labels_test.shape)

In [0]:
labels_test[climbstairs_group_imgs_test:run_group_imgs_test] = 1. 

In [0]:
run_img_test = imgs_test[0, ...]       
for i in range(3):        
        plt.imshow(run_img_test[..., i], cmap='gray')        
        plt.grid(False) 
        plt.show()

In [0]:
os.chdir('..') 
os.chdir('ride_bike_test') 

In [0]:
imgs_test = np.concatenate((imgs_test, video_loader()), 0) 

In [0]:
ridebike_group_imgs_test = imgs_test.shape[0]
print(imgs_test.shape)

In [0]:
labels_test = np.zeros([ridebike_group_imgs_test, ]) 
print(labels_test.shape)

In [0]:
labels_test[run_group_imgs_test:ridebike_group_imgs_test] = 2. 

In [0]:
ride_bike_img_test = imgs_test[0, ...]        
for i in range(3):        
        plt.imshow(ride_bike_img_test[..., i], cmap='gray')        
        plt.grid(False) 
        plt.show()

In [0]:
print(imgs_test.shape)

In [0]:
from tflearn.data_utils import to_categorical 
labels_test = to_categorical(labels_test, 3)  

In [0]:
print(labels_test.shape)

# Training V. Testing

In [0]:
print(imgs_train.shape, labels_train.shape, imgs_test.shape, labels_test.shape)  
print(labels_train[:10, ...])  

In [0]:
input_layer = input_data([None, 240, 320, 3])


conv1 = conv_2d(input_layer, 32, 7, strides=2, activation='relu')


pool1 = max_pool_2d(conv1, 3, 2)


residual_block1 = residual_block(pool1, 
                                 32, 
                                 3, 
                                 3) 


residual_block2 = residual_block(residual_block1, 
                                 64, 
                                 3, 
                                 4, 
                                 2) 


residual_block3 = residual_block(residual_block2,
                                 128,
                                 3,
                                 6, 
                                 2)


residual_block4 = residual_block(residual_block3,
                                 256,
                                 3,
                                 3,
                                 2)


gap = global_avg_pool(residual_block4)


output_layer = fully_connected(gap, 3, activation='softmax')

In [0]:
network = regression(output_layer, optimizer='adam', 
                     loss='categorical_crossentropy', learning_rate=.0001)

In [0]:
model = tflearn.DNN(network, tensorboard_verbose=2)

In [0]:
install_tensorboard_dep()
start_tensorboard()

In [0]:
model.fit(imgs_train, 
          labels_train, 
          n_epoch=25, 
          shuffle=True,  
          validation_set=(imgs_test, labels_test),  
          show_metric=True, 
          batch_size=20,  
          run_id='three_group_diff')  

model.save('three_group_diff_ResNet')